In [1]:
import h5py
import torch
import transformers
from tqdm import tqdm
from transformers import AutoModelForMaskedLM, AutoTokenizer,AutoModel
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
sys.path.append('../data_generation')
import utils
import numpy as np

/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datalen = '512'
file = h5py.File("../data/CAGI/"+datalen+"/CAGI_onehot.h5", "r")
alt = file['alt']
ref = file['ref']

## GPN claim zero shot learning with log p(alt)/p(ref)

Need to mask the center nuceltide of each seq and get the p value

In [3]:
import gpn
import gpn.model
model = AutoModelForMaskedLM.from_pretrained("../model/GPN_human/checkpoint-2000000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("../model/GPN_human/checkpoint-2000000")
model.eval();

In [5]:
N, L, A = alt.shape
mut_i = int(L/2-1)
cagi_llr=[]
for i in tqdm(range(N)):
    ref_a = np.argmax(ref[i][mut_i])+2
    alt_a = np.argmax(alt[i][mut_i])+2
    seq = utils.onehot_to_seq([ref[i]])
    token = tokenizer.batch_encode_plus(seq)['input_ids'][0]
    token[mut_i] = tokenizer.mask_token_id
    output = model(torch.tensor([token]).to('cuda')).logits.cpu().detach().numpy()
    llr = output[0][mut_i][alt_a] - output[0][mut_i][ref_a]
    cagi_llr.append(llr)


  0%|          | 0/18442 [00:00<?, ?it/s]/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 18442/18442 [02:21<00:00, 130.55it/s]


In [18]:
output = h5py.File('../data/CAGI/'+'cagi_'+datalen+'_'+'gpn.h5', 'w')
output.create_dataset('llr', data=np.array(cagi_llr))
output.close()

## Visulization


In [1]:
import h5py
import pandas as pd
import numpy as np
from operator import itemgetter
import seaborn as sns
import matplotlib.pyplot as plt 
import scipy.stats as stats
datalen='512'

In [2]:
cagi_result = h5py.File('../data/CAGI/'+'cagi_'+datalen+'_'+'gpn.h5', 'r')
cagi_llr = cagi_result['llr'][()]

In [3]:
cagi_df = pd.read_csv('../data/CAGI/'+datalen+'/final_cagi_metadata.csv',
                      index_col=0).reset_index()
target = cagi_df['6'].values.tolist()
exp_list = cagi_df['8'].unique()

In [4]:
perf = []
sanity_check = 0
for exp in ['LDLR','SORT1','F9','PKLR']:
    sub_df = cagi_df[cagi_df['8'] == exp]
    sanity_check += len(sub_df)
    exp_target = np.array(target)[sub_df.index.to_list()]
    exp_pred = np.squeeze(cagi_llr)[sub_df.index.to_list()]
    exp_target = np.absolute(exp_target)
    exp_pred = np.absolute(exp_pred)
    print(exp)
    perf.append(stats.pearsonr(exp_pred,exp_target)[0])
    print(stats.pearsonr(exp_pred,exp_target)[0])

LDLR
-0.07866246778179416
SORT1
-0.001525750065541454
F9
0.07505647712949828
PKLR
0.03675438661873984


In [5]:
(-0.07866-0.00152+0.07505)/3

-0.0017099999999999939

## CAGI embedding generation for lenti model

In [2]:
import gpn
import gpn.model

datalen = '230'
file = h5py.File("../data/CAGI/"+datalen+"/CAGI_onehot.h5", "r")
alt = file['alt']
ref = file['ref']

model = AutoModel.from_pretrained("../model/GPN_human/checkpoint-2000000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("../model/GPN_human/checkpoint-2000000")
model.eval();

Some weights of the model checkpoint at ../model/GPN_human/checkpoint-2000000 were not used when initializing ConvNetModel: ['cls.decoder.2.weight', 'cls.decoder.0.bias', 'cls.decoder.0.weight', 'cls.decoder.2.bias', 'cls.decoder.3.bias', 'cls.decoder.3.weight']
- This IS expected if you are initializing ConvNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ConvNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
batch_size = 128
N, L, A = alt.shape
ref_out = []
alt_out = []
for i in tqdm(range(0,N,batch_size)):
    ref_seq = ref[i:i+batch_size]
    alt_seq = alt[i:i+batch_size]
    ref_seq = utils.onehot_to_seq(ref_seq)
    alt_seq = utils.onehot_to_seq(alt_seq)
    ref_token = tokenizer.batch_encode_plus(ref_seq)['input_ids']
    alt_token = tokenizer.batch_encode_plus(alt_seq)['input_ids']

    ref_out.extend(model(torch.tensor(ref_token).to('cuda')).last_hidden_state.cpu().detach().numpy())
    alt_out.extend(model(torch.tensor(alt_token).to('cuda')).last_hidden_state.cpu().detach().numpy())


  0%|          | 0/145 [00:00<?, ?it/s]/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 145/145 [00:34<00:00,  4.23it/s]


In [6]:
output = h5py.File('../data/CAGI/230_embed/'+'gpn.h5', 'w')
output.create_dataset('ref', data=np.array(ref_out))
output.create_dataset('alt', data=np.array(alt_out))
output.close()